In [1]:
cd ~/git/thinking_like_transformers

/Users/ellenar/git/thinking_like_transformers


In [2]:
#@title Imports
import jax
import numpy as np

# The default of float16 can lead to discrepancies between outputs of
# the compiled model and the RASP program.
jax.config.update('jax_default_matmul_precision', 'float32')

from tracr_utils.lib.tracr.compiler import compiling, assemble
from tracr.rasp import rasp
import torch
import torch.nn as nn

In [3]:
from model.transformer_model import TransformerModel
from inference.algorithms import *
from inference.load_weights import extract_config, build_state_dict
from utils import register_hooks

In [4]:
input_= ['bos',3,2,2,2]
algorithm = hist()
bos = "bos"
compiled_model = compiling.compile_rasp_to_model(
    algorithm,
    vocab={1,2,3,4},
    max_seq_len=5,
    compiler_bos=bos,
)
decoded_output_1 = compiled_model.apply(input_)

In [5]:
model_config = extract_config(compiled_model)
state_dict = build_state_dict(compiled_model)
torch_model = TransformerModel(model_config).eval()
activations_dict = {}
register_hooks(torch_model, activations_dict)
torch_model.load_state_dict(state_dict)
   
x = compiled_model.input_encoder.encode(input_)
out = torch_model(x)

In [6]:
decode_fn = get_decode_fn('hist', model_config)

In [7]:
decoded_output_2 = decode_fn(out, compiled_model)
decoded_output_2 == decoded_output_1.decoded

True

In [8]:
decoded_output_1.decoded

['bos', 1, 3, 3, 3]